# Initial data visualization to see if everything is looking good

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import gc

cwd = os.getcwd()
base_dir = os.path.dirname(cwd)
data_dir = base_dir + "/data/octomag_data/data"
src_dir = base_dir + "/src"

print("data dir: ", data_dir)
print("src dir: ", src_dir)


sys.path.insert(0, src_dir)

from data_analysis import load_data, convert_frames, plot_positions, plot_quiver_slice, plot_quiver_3d, correct_sensor_bias

## Load raw data

In [ ]:
df_og = load_data(data_dir)

df_og_converted = convert_frames(df_og)
em_cols = [col for col in df_og.columns if col.startswith("em_")]

In [ ]:
df_og

## Check sensor dc offset

In [ ]:
eps = 1e-3
mask_zeros = df_og_converted[em_cols].abs().lt(eps).all(axis=1)
B_zeros_corrected = df_og_converted.loc[mask_zeros, ['Bx','By','Bz']]

B_zeros_corrected.hist(bins=100, layout=(1,3), figsize=(9,3), sharex=False, sharey=False)
plt.tight_layout(); plt.show()

## Correct for DC offset

The corrected 0 currents distribution is already much better. Note the current distribution and then check the quiver plots below also for 0 currents. It actually makes sense :)

In [ ]:
del df_og_converted
gc.collect()

df_bias_corrected = correct_sensor_bias(df_og, bias_csv=data_dir + "/sensor_bias.csv")

del df_og
gc.collect()

df_bias_corrected_converted = convert_frames(df_bias_corrected)

In [ ]:
eps = 1e-3
mask_zeros = df_bias_corrected_converted[em_cols].abs().lt(eps).all(axis=1)
B_zeros_corrected = df_bias_corrected_converted.loc[mask_zeros, ['Bx','By','Bz']]

B_zeros_corrected.hist(bins=100, layout=(1,3), figsize=(9,3), sharex=False, sharey=False)
plt.tight_layout(); plt.show()

## Let's use this corrected dataset

In [ ]:
df = df_bias_corrected_converted

del df_bias_corrected
gc.collect()

df

## Plot positions of corrected data

In [ ]:
plot_positions(df, point_size=1.5, max_points=1e6)

In [ ]:
df_reduced = df[(df["base_x"] == 0) &
                (df["base_y"] == 0) &
                (df["z_offset"] == 0)]

plot_positions(df_reduced, point_size=3.0)

## Plot quiver

In [ ]:
currents = {
    'em_0': 4.0,
    'em_1': 0.0,
    'em_2': 0.0,
    'em_3': 0.0,
    'em_4': 0.0,
    'em_5': 0.0,
    'em_7': 0.0,
    'em_8': 0.0,
}                           # A

normal_direction = "z"      # "x", "y", or "z"    
normal_coord = -0.0        # m
normal_margin = 0.002       # m
current_tolerance = 1e-3    # A
scale = 0.3                   # mm/mT




_ = plot_quiver_slice(
    df,
    currents,
    normal_direction,
    normal_coord,
    normal_margin,
    current_tolerance,
    scale,
    Nmax=100000              # downsample for plotting
)

In [ ]:
currents = {
    'em_0': 0.0,
    'em_1': 1.0,
    'em_2': 0.0,
    'em_3': 0.0,
    'em_4': 0.0,
    'em_5': 0.0,
    'em_7': 0.0,
    'em_8': 0.0,
}                           # A

current_tolerance = 1e-1    # A
scale = 1                   # mm/mT

plot_quiver_3d(
    df,
    currents,
    current_tolerance,
    scale,
    Nmax=5000,              # max number of points
    arrow_linewidth = 2.0
)